# Setup

In [1]:
# my virtual environments are rarely properly connected to jupyter so this fixes that. 
# you prolly won't need this cell but running it won't hurt anything either
import sys
import os
current_dir = os.getcwd()  # Get the current working directory
venv_dir = os.path.join(current_dir, 'venv') 
python_version = str(sys.version_info.major) + '.' + str(sys.version_info.minor)
site_packages_path = os.path.join(venv_dir, 'lib', 'python' + python_version, 'site-packages')
sys.path.append(site_packages_path) 

# Instantiate a brand new model

In [2]:
# config file
from config import ModelConfig, TrainConfig
cfg = ModelConfig()
tcfg = TrainConfig()
print(cfg, '\n\n', tcfg)

# import the tokenizer specified by cfg
from tools import import_from_nested_path
imported_objects = import_from_nested_path(['tokenizers', cfg.tokenizer], 'tokenizer', ['get_tokenizer'])
get_tokenizer = imported_objects.get('get_tokenizer')
tokenizer = get_tokenizer(size = cfg.vocab_len)

# model modules
from modules.model import Model
model = Model(cfg).to(cfg.device)

# print the number of parameters in the model
print(f'\n{sum(p.numel() for p in model.parameters())/1e3}K parameters\n')
print(model)

ModelConfig(dim=96, device='cpu', dropout_rate=0.1, weight_tying=True, tokenizer='bpe_v2', vocab_len=1024, num_layers=6, second_resid_norm=False, mlp_hidden_mult=4, mlp_bias=False, mlp_nonlinearity='SiLU', mlp_gated=True, num_q_heads=2, num_kv_heads=1, head_dim=48, theta=10000, max_seq_len=512, ca_num_q_heads=2, ca_num_kv_heads=1, ca_head_dim=48, scale_first_resid=True, norm_type='RMSNorm', norm_affine=True, norm_bias=True, eps=1e-06, pool_type='sum', pre_pool_norm=True, pool_output_linear=False, pool_bias=False, compress_freq='constant', compress_freq_n=1, fs_mult=3, fs_periods=3, fs_loss_lambda=1.0, max_batch_size=1) 

 TrainConfig(weight_decay=0.05, batch_size=32, max_iters=2000, eval_interval=20, eval_samples=1, checkpoint_interval=None, lr_init=1e-06, lr_max=0.1, lr_min=0.001, warmup_iters=20, final_flat_iters=200, anneal_type='cos', num_restarts=3, T_mult=2)

793.632K parameters

Model(
  (token_embedder): Embedding(1027, 96)
  (body_layers): ModuleList(
    (0-2): 3 x Layer(
   

# Training

In [3]:
import torch
from tools import get_data_loader
from train import scheduler_lambda, train

optimizer = torch.optim.AdamW(model.parameters(), lr = tcfg.lr_max, weight_decay = tcfg.weight_decay)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=scheduler_lambda)

train_data_loader = get_data_loader(batch_size=tcfg.batch_size, split='train')
test_data_loader = get_data_loader(batch_size=tcfg.batch_size, split='validation')

In [4]:
if False: # set to true if you'd like to see a graph of the learning rate schedule
    import matplotlib.pyplot as plt
    
    # Generate learning rate values
    lrs = [scheduler_lambda(i) for i in range(tcfg.max_iters)]
    
    # Plot the learning rates
    plt.figure(figsize=(10, 5))
    plt.plot(lrs, label='Learning Rate')
    plt.title('Learning Rate Schedule')
    plt.xlabel('Iteration')
    plt.ylabel('Learning Rate')
    plt.grid(True)
    plt.legend()
    plt.show()

In [5]:
model, optimizer, log_data = train(
    model, 
    tokenizer, 
    cfg, 
    optimizer,
    scheduler,
    tcfg, 
    train_data_loader,
    test_data_loader,
    #log_data: list = None, 
    #detect_anomoly = False # use if you're getting crazy errors about a the gradient being broken
)

  0%|                                        | 1/2000 [00:08<4:31:01,  8.14s/it]

step 0000: lr 0.000000, train loss 9.3430, val loss 9.3450, ppl 11442, time elapsed: 4.10 seconds


  1%|▍                                      | 21/2000 [01:32<2:54:44,  5.30s/it]

step 0020: lr 0.010000, train loss 4.8031, val loss 4.8190, ppl 124, time elapsed: 88.75 seconds


  2%|▊                                      | 41/2000 [02:57<2:46:55,  5.11s/it]

...

step 1960: lr 0.000100, train loss 0.1867, val loss 0.1691, ppl 1, time elapsed: 7654.42 seconds


 99%|████████████████████████████████████▋| 1981/2000 [2:08:52<01:28,  4.66s/it]

step 1980: lr 0.000100, train loss 0.1774, val loss 0.1733, ppl 1, time elapsed: 7729.36 seconds


100%|█████████████████████████████████████| 2000/2000 [2:10:04<00:00,  3.90s/it]

step 1999: lr 0.000100, train loss 0.1828, val loss 0.1796, ppl 1, time elapsed: 7800.74 seconds


# inference test before you decide to save it

In [6]:
from inference import generate
prompt = "Once upon a time"
model.eval()
output = generate(
    prompt, 
    model, 
    tokenizer,
    #max_gen_len = 512,
    temperature = 0.9,
    #memory_saver_div = 8,
    #top_p = 0.9,
    #top_k = 32,
)
model.train()
print(output)

Once upon a time His gria's bear cao bye cat an an o ain bed ins al alve an high ne e wasb."
.
"Myin."Sogbbit, Sueeip ho modus's toat. walk waspushing an and a to.ify an up o own frog, with birds....s Suey's anutedyy became, and it to inge, Sue.
One.
? to oue of an and it wass ha mor startedying to togey. Sue, Spot! to fo a by walkaa was. insideyed at an an an an anke a arod! tolwre e alreap fow!? By wo co.
" an and toys, uning dog to?ist are to in upp Have. that his o.s to. Ak. an and Mom's anarat o ow al. Kitty's an an and ideas an bua.yed. Max an ind eatin foing kids to to.ped oI be with bluey thingss donn't, an in ore, love playing to mw be. many fotionub ns than an and warm to quess an an Gyo be as from an and ne e givenes o aroed animals's to pa. Mia aedffila to playingw soar that his lodyy.Mom wasing always.Mom toie. Sue wantgs.inseo saw to toss. many guesMom to thingsow into give o ow haveth chalingly that, to MAs by by oal. walk deorcalus heeded an an and it had an uns. it so 

# Saving your final model
if `tcfg.checkpoint_interval != None` then checkpoints have already been saved

you DO still need to do this even if you had been saving checkpoints; the final state has not yet been saved

In [7]:
from tools import save_model
save_model(model, cfg, tcfg, log_data)